In [21]:
import os 
import numpy as np
import tensorflow as tf

from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, Convolution2D, core, BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers.core import Lambda
import keras

from keras import backend as K
from keras.backend import binary_crossentropy

# config = tf.ConfigProto()
# config.gpu_options.allocator_type ='BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [22]:
tag = 'highway'

In [23]:
%run data.ipynb

In [24]:
mydata = dataProcess(256,256, tag=tag)
mydata.create_train_data()
mydata.create_test_data()

------------------------------
Creating training images...
------------------------------
3450
loading done
Saving to .npy files done.
------------------------------
Creating test images...
------------------------------
3450
loading done
Saving to imgs_test.npy files done.


In [25]:
def slice_batch(x, n_gpus, part):
    """
    Divide the input batch into [n_gpus] slices, and obtain slice no.
    [part].

    i.e. if len(x)=10, then slice_batch(x, 2, 1) will return x[5:].

    """

    sh = K.shape(x)

    L = sh[0] / n_gpus
    L = tf.cast(L, tf.int32)

    if part == n_gpus - 1:

        return x[part*L:]

    return x[part*L:int(part+1)*L]

def to_multi_gpu(model, n_gpus=4):

    """Given a keras [model], return an equivalent model which parallelizes

    the computation over [n_gpus] GPUs.



    Each GPU gets a slice of the input batch, applies the model on that
    slice

    and later the outputs of the models are concatenated to a single
    tensor,

    hence the user sees a model that behaves the same as the original.

    """

    with tf.device('/cpu:0'):

        x = Input(model.input_shape[1:], name=model.input_names[0])


    towers = []

    for g in range(n_gpus):

        with tf.device('/gpu:' + str(g)):

            slice_g = Lambda(slice_batch, lambda shape: shape, arguments={'n_gpus':n_gpus, 'part':g})(x)

            towers.append(model(slice_g))


    with tf.device('/cpu:0'):

        merged = merge(towers, mode='concat', concat_axis=0)

    new_model = Model(input=[x], output=merged)
    
    funcType = type(model.save)
    def new_save(self_,filepath, overwrite=True):
        model.save(filepath, overwrite)
    new_model.save=funcType(new_save, new_model)
   
    return new_model

In [26]:
smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + binary_crossentropy(y_pred, y_true)


In [27]:
class myUnet(object):

    def __init__(self, tp = 'highway', img_rows = 256, img_cols = 256):

        self.tp = tp
        self.type = tp
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):

        mydata = dataProcess(self.img_rows, self.img_cols, tag = self.type)
        imgs_train, imgs_mask_train = mydata.load_train_data()
        imgs_test = mydata.load_test_data()
        return imgs_train, imgs_mask_train, imgs_test

    #Define the neural network
    def get_unet(self):
        inputs = Input((256, 256, 1))
        #
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(inputs)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv1)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        #
        conv2 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool1)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same', kernel_initializer = 'he_normal')(conv2)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        #
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool2)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv3)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        #
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool3)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv4)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
        drop4 = Dropout(0.5)(conv4)

        #
        up1 = merge([UpSampling2D(size=(2, 2))(conv4), conv3], mode='concat', concat_axis=-1)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up1)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv7)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        #
        up3 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=-1)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(up3)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(conv8)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        #
        up4 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=-1)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up4)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(2, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        #
        conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

        model = Model(input=inputs, output=conv10)

        model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])
#         model = to_multi_gpu(model, n_gpus=2)
#         model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

        return model


    def train(self):
        print("loading train data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading train data done")
        model = self.get_unet()
        print("got unet")

        model_checkpoint = ModelCheckpoint('model_weights/sentinel_unet_%s.hdf5' % self.tp, monitor='loss',verbose=1, save_best_only=True, mode='min')
        print('Fitting model...')
        model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=50, verbose=1, validation_split=0.3, shuffle=True, callbacks=[model_checkpoint])

    def predict(self):
        print("loading test data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading test data done")
        model = self.get_unet()
        print("got unet")
        
        model.load_weights('model_weights/sentinel_unet_%s.hdf5' % self.tp)
        model.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,
                      metrics=['binary_crossentropy', jaccard_coef_int])
        print('Predicting...')
        imgs_mask_test = model.predict(imgs_test, batch_size=8, verbose=1)
        np.save('npy/imgs_mask_test_%s.npy' % self.tp, imgs_mask_test)

    def save_img(self):

        print("Array to images")
        imgs = np.load('npy/imgs_mask_test_%s.npy' % self.tp)
        imgs_locations = np.load('npy/imgs_test_file_locations.npy')
        for i in range(imgs.shape[0]):
            img = imgs[i]
            img[img > 0.5] = 1
            img[img <= 0.5] = 0
            img = array_to_img(img)
            img_location = imgs_locations[i]
            img_location = '/'.join(img_location.split('/')[:-1]) + '/' + img_location.split('/')[-1].split('.')[0] + '_pred_%s.png' % self.tp

            img.save(img_location)

In [ ]:
myunet = myUnet(tag)
myunet.train()
myunet.predict()
myunet.save_img()

loading train data
------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------
loading train data done


/usr/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_normal", padding="same")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), kernel_initializer="he_normal", padding="same")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), kernel_initializer="he_normal", padding="same")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `BatchNormalization` call to the Kera

got unet
Fitting model...
Train on 2415 samples, validate on 1035 samples
Epoch 1/50
2415/2415 [==============================] - 270s 112ms/step - loss: 4.1634 - binary_crossentropy: 0.2014 - jaccard_coef_int: 0.0352 - val_loss: 5.8581 - val_binary_crossentropy: 0.1666 - val_jaccard_coef_int: 0.1428
Epoch 2/50
2415/2415 [==============================] - 262s 109ms/step - loss: 3.5315 - binary_crossentropy: 0.3013 - jaccard_coef_int: 0.0562 - val_loss: 5.2631 - val_binary_crossentropy: 0.2182 - val_jaccard_coef_int: 0.1538
Epoch 3/50
2415/2415 [==============================] - 262s 109ms/step - loss: 3.3742 - binary_crossentropy: 0.3308 - jaccard_coef_int: 0.0635 - val_loss: 4.6139 - val_binary_crossentropy: 0.2271 - val_jaccard_coef_int: 0.1576
Epoch 4/50
2415/2415 [==============================] - 262s 109ms/step - loss: 3.2158 - binary_crossentropy: 0.3514 - jaccard_coef_int: 0.0735 - val_loss: 4.3886 - val_binary_crossentropy: 0.2666 - val_jaccard_coef_int: 0.1742
Epoch 5/50
196